# Analisi dei dati raccolti

## Setup

In [21]:
import pandas as pd

In [22]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconprfsv12

In [23]:
path = "../asconprfsv12/data/{}.csv"

### Armv7m

In [24]:
with open(path.format("armv7m"), "r") as f:
    aprfs_armv7m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [25]:
# Generazione tag
data = aprfs_armv7m["0B-A"]

print_data(data)

0B-A
24    877
25     83
26     40
Name: count, dtype: int64

Massimo: 26
Minimo: 24
Media: 24.163
Varianza: 0.21664764764764774


In [26]:
# Verifica tag
data = aprfs_armv7m["0B-V"]

print_data(data)

0B-V
56    959
57     40
55      1
Name: count, dtype: int64

Massimo: 57
Minimo: 55
Media: 56.039
Varianza: 0.03951851851851851


#### PT 8 byte

In [27]:
# Generazione tag
data = aprfs_armv7m["8B-A"]

print_data(data)

8B-A
25    960
26     40
Name: count, dtype: int64

Massimo: 26
Minimo: 25
Media: 25.04
Varianza: 0.038438438438438444


In [28]:
# Verifica tag
data = aprfs_armv7m["8B-V"]

print_data(data)

8B-V
57    960
58     40
Name: count, dtype: int64

Massimo: 58
Minimo: 57
Media: 57.04
Varianza: 0.038438438438438444


#### PT 16 byte

In [29]:
# Generazione tag
data = aprfs_armv7m["16B-A"]

print_data(data)

16B-A
25    959
26     41
Name: count, dtype: int64

Massimo: 26
Minimo: 25
Media: 25.041
Varianza: 0.03935835835835836


In [30]:
# Verifica tag
data = aprfs_armv7m["16B-V"]

print_data(data)

16B-V
57    957
59     40
58      3
Name: count, dtype: int64

Massimo: 59
Minimo: 57
Media: 57.083
Varianza: 0.15626726726726725


### Armv7m_small

In [31]:
with open(path.format("armv7m_small"), "r") as f:
    aprfs_small = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [32]:
# Generazione tag
data = aprfs_small["0B-A"]

print_data(data)

0B-A
22    836
21    164
Name: count, dtype: int64

Massimo: 22
Minimo: 21
Media: 21.836
Varianza: 0.1372412412412413


In [33]:
# Verifica tag
data = aprfs_small["0B-V"]

print_data(data)

0B-V
47    958
49     40
48      2
Name: count, dtype: int64

Massimo: 49
Minimo: 47
Media: 47.082
Varianza: 0.15543143143143145


#### PT 8 byte

In [34]:
# Generazione tag
data = aprfs_small["8B-A"]

print_data(data)

8B-A
22    999
21      1
Name: count, dtype: int64

Massimo: 22
Minimo: 21
Media: 21.999
Varianza: 0.0010000000000000005


In [35]:
# Verifica tag
data = aprfs_small["8B-V"]

print_data(data)

8B-V
48    959
49     41
Name: count, dtype: int64

Massimo: 49
Minimo: 48
Media: 48.041
Varianza: 0.039358358358358346


#### PT 16 byte

In [36]:
# Generazione tag
data = aprfs_small["16B-A"]

print_data(data)

16B-A
22    999
21      1
Name: count, dtype: int64

Massimo: 22
Minimo: 21
Media: 21.999
Varianza: 0.0010000000000000005


In [37]:
# Verifica tag
data = aprfs_small["16B-V"]

print_data(data)

16B-V
48    957
50     40
49      3
Name: count, dtype: int64

Massimo: 50
Minimo: 48
Media: 48.083
Varianza: 0.15626726726726725


### Bi32_armv7m

In [38]:
with open(path.format("bi32_armv7m"), "r") as f:
    aprfs_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [39]:
# Generazione tag
data = aprfs_bi32["0B-A"]

print_data(data)

0B-A
23    953
25     40
24      7
Name: count, dtype: int64

Massimo: 25
Minimo: 23
Media: 23.087
Varianza: 0.1595905905905907


In [40]:
# Verifica tag
data = aprfs_bi32["0B-V"]

print_data(data)

0B-V
50    958
52     40
51      2
Name: count, dtype: int64

Massimo: 52
Minimo: 50
Media: 50.082
Varianza: 0.15543143143143145


#### PT 8 byte

In [41]:
# Generazione tag
data = aprfs_bi32["8B-A"]

print_data(data)

8B-A
25    719
24    241
26     40
Name: count, dtype: int64

Massimo: 26
Minimo: 24
Media: 24.799
Varianza: 0.24083983983983984


In [42]:
# Verifica tag
data = aprfs_bi32["8B-V"]

print_data(data)

8B-V
53    956
54     41
52      3
Name: count, dtype: int64

Massimo: 54
Minimo: 52
Media: 53.038
Varianza: 0.04259859859859861


#### PT 16 byte

In [43]:
# Generazione tag
data = aprfs_bi32["16B-A"]

print_data(data)

16B-A
26    717
25    244
27     39
Name: count, dtype: int64

Massimo: 27
Minimo: 25
Media: 25.795
Varianza: 0.24121621621621625


In [44]:
# Verifica tag
data = aprfs_bi32["16B-V"]

print_data(data)

16B-V
54    956
55     42
53      2
Name: count, dtype: int64

Massimo: 55
Minimo: 53
Media: 54.04
Varianza: 0.04244244244244244


### Ref

In [45]:
with open(path.format("ref"), "r") as f:
    aprfs_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [46]:
# Generazione tag
data = aprfs_ref["0B-A"]

print_data(data)

0B-A
77    718
78    242
79     40
Name: count, dtype: int64

Massimo: 79
Minimo: 77
Media: 77.322
Varianza: 0.29861461461461464


In [47]:
# Verifica tag
data = aprfs_ref["0B-V"]

print_data(data)

0B-V
162    839
163    159
161      2
Name: count, dtype: int64

Massimo: 163
Minimo: 161
Media: 162.157
Varianza: 0.13648748748748743


#### PT 8 byte

In [48]:
# Generazione tag
data = aprfs_ref["8B-A"]

print_data(data)

8B-A
83    919
84     81
Name: count, dtype: int64

Massimo: 84
Minimo: 83
Media: 83.081
Varianza: 0.07451351351351351


In [49]:
# Verifica tag
data = aprfs_ref["8B-V"]

print_data(data)

8B-V
173    838
174    162
Name: count, dtype: int64

Massimo: 174
Minimo: 173
Media: 173.162
Varianza: 0.13589189189189188


#### PT 16 byte

In [50]:
# Generazione tag
data = aprfs_ref["16B-A"]

print_data(data)

16B-A
88    915
89     84
90      1
Name: count, dtype: int64

Massimo: 90
Minimo: 88
Media: 88.086
Varianza: 0.08068468468468469


In [51]:
# Verifica tag
data = aprfs_ref["16B-V"]

print_data(data)

16B-V
184    839
185    161
Name: count, dtype: int64

Massimo: 185
Minimo: 184
Media: 184.161
Varianza: 0.1352142142142142
